# 🏥 SimuPaciente UMH - Demo en Google Colab

**Universidad Miguel Hernández de Elche**

Sistema de simulación de pacientes virtuales con voz para práctica de entrevistas clínicas.

---

## 📋 Instrucciones

1. **Ejecuta las celdas en orden** (Shift + Enter)
2. **Proporciona tu API key de OpenAI** cuando se te solicite
3. **Accede a la URL de ngrok** que se generará al final
4. **¡Comienza a practicar!**

⚠️ **IMPORTANTE:** Este notebook usa recursos de Colab gratuitos. La sesión se cerrará automáticamente después de ~12 horas de inactividad.

---

## 🔧 Paso 1: Instalación de Dependencias

Instalamos Python, Node.js y todas las dependencias necesarias.

In [ ]:
%%bash
echo "📦 Instalando dependencias del sistema..."

# Actualizar sistema
apt-get update -qq

# Instalar Node.js 18.x
curl -fsSL https://deb.nodesource.com/setup_18.x | bash -
apt-get install -y nodejs

# Verificar versiones
echo "✅ Node.js version: $(node --version)"
echo "✅ npm version: $(npm --version)"
echo "✅ Python version: $(python3 --version)"

## 📥 Paso 2: Clonar Repositorio

Descargamos el código desde GitHub.

In [ ]:
%%bash
echo "📥 Clonando repositorio desde GitHub..."

# Ir a /content primero
cd /content

# Limpiar si existe
rm -rf ECOE

# Clonar repo
git clone https://github.com/marcosbenghezala/ECOE.git ECOE

echo "✅ Repositorio clonado exitosamente"
ls -la /content/ECOE/

## 🐍 Paso 3: Instalar Dependencias Python

Instalamos las librerías de Python necesarias para el backend.

In [ ]:
%%bash
cd /content/ECOE/simulador

echo "🐍 Instalando dependencias Python..."
pip install -q -r requirements.txt

echo "✅ Dependencias Python instaladas"

## ⚛️ Paso 4: Build del Frontend

Compilamos el frontend React.

In [ ]:
%%bash
cd /content/ECOE/simulador/frontend

echo "⚛️ Instalando dependencias de Node.js..."
npm install --silent

echo "🔨 Building frontend..."
npm run build

echo "✅ Frontend compilado exitosamente"
ls -la /content/ECOE/simulador/frontend/dist/

## 🔑 Paso 5: Configuración del Proxy Server

**AUTOMÁTICO** - Los estudiantes NO necesitan hacer nada aquí.

La API key de OpenAI está configurada en el servidor proxy (oculta y segura).

📖 **Instrucciones para el profesor:** Ver `DEPLOYMENT_GUIDE.md` para desplegar el proxy.

In [ ]:
import os

# ============================================
# ✅ CONFIGURACIÓN AUTOMÁTICA VÍA PROXY SERVER
# ============================================
#
# Tu API key de OpenAI está en el servidor proxy
# Los estudiantes NO necesitan configurar nada
#
# ============================================

# URL del proxy server (desplegado en Render.com)
# ⚠️ IMPORTANTE: Reemplaza esta URL con la tuya después de desplegar
PROXY_URL = "REEMPLAZAR_CON_TU_URL_DE_RENDER"

# Verificar que se configuró la URL
if PROXY_URL == "REEMPLAZAR_CON_TU_URL_DE_RENDER":
    print("⚠️" * 30)
    print("")
    print("❌ ERROR: Necesitas configurar la URL del proxy server")
    print("")
    print("📖 Sigue estas instrucciones:")
    print("")
    print("1. Despliega el proxy server en Render.com (100% gratis)")
    print("   - Ver guía completa en: DEPLOYMENT_GUIDE.md")
    print("   - https://github.com/marcosbenghezala/ECOE/blob/main/DEPLOYMENT_GUIDE.md")
    print("")
    print("2. Copia la URL que Render te da")
    print("   - Ejemplo: https://simu-paciente-umh-proxy.onrender.com")
    print("")
    print("3. Reemplaza 'REEMPLAZAR_CON_TU_URL_DE_RENDER' arriba con tu URL")
    print("")
    print("4. Ejecuta esta celda de nuevo")
    print("")
    print("⚠️" * 30)
    raise ValueError("Proxy URL not configured")

# Configurar para usar el proxy
os.environ['PROXY_URL'] = PROXY_URL

print("="*70)
print("✅ CONFIGURACIÓN COMPLETADA AUTOMÁTICAMENTE")
print("="*70)
print(f"\n🔒 Proxy server: {PROXY_URL}")
print("")
print("📝 Información:")
print("   - La API key de OpenAI está en el servidor proxy (oculta)")
print("   - Los estudiantes NO pueden ver ni copiar la API key")
print("   - El uso está controlado a través del proxy")
print("   - Servidor desplegado en Render.com (gratis)")
print("")
print("👉 Continúa ejecutando las siguientes celdas")
print("="*70)

## 🌐 Paso 6: Instalar y Configurar ngrok

ngrok nos permite exponer el servidor local a Internet de forma segura.

In [ ]:
%%bash
echo "🌐 Instalando ngrok..."

# Descargar ngrok
wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
tar xzf ngrok-v3-stable-linux-amd64.tgz
chmod +x ngrok
mv ngrok /usr/local/bin/

echo "✅ ngrok instalado"
ngrok version

## 🚀 Paso 7: Iniciar Servidor

**¡Última celda!** Esto iniciará el servidor y creará una URL pública.

⚠️ **IMPORTANTE:**
- Esta celda quedará ejecutándose (verás [*] a la izquierda)
- **NO la detengas** mientras uses la aplicación
- Busca la URL de ngrok en la salida (https://xxxx-xx-xxx.ngrok-free.app)
- Copia esa URL y ábrela en una nueva pestaña

In [ ]:
import subprocess
import time
import threading
import requests
from IPython.display import display, HTML
import re

print("🚀 Iniciando SimuPaciente UMH...")
print("="*70)

# Cambiar al directorio del simulador
import os
os.chdir('/content/ECOE/simulador')

# PASO 1: Limpiar procesos anteriores
print("\n🧹 Limpiando procesos anteriores...")
subprocess.run(['pkill', '-f', 'colab_server.py'], stderr=subprocess.DEVNULL)
subprocess.run(['pkill', '-f', 'ngrok'], stderr=subprocess.DEVNULL)
subprocess.run(['fuser', '-k', '8080/tcp'], stderr=subprocess.DEVNULL)
time.sleep(2)
print("✅ Procesos anteriores limpiados")

# PASO 2: Iniciar ngrok PRIMERO (en background, capturando output)
print("\n🌐 Iniciando túnel ngrok...")
ngrok_log_file = open('/tmp/ngrok.log', 'w')
ngrok_process = subprocess.Popen(
    ['ngrok', 'http', '8080', '--log=stdout'],
    stdout=ngrok_log_file,
    stderr=subprocess.STDOUT,
    text=True
)

# Dar tiempo a ngrok para iniciar
print("⏳ Esperando a que ngrok establezca el túnel...")
time.sleep(10)

# PASO 3: Iniciar servidor Flask
print("\n🏥 Iniciando servidor Flask...")
server_process = subprocess.Popen(
    ['python3', 'colab_server.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

# Esperar a que el servidor esté listo
print("⏳ Esperando a que el servidor esté listo...")
time.sleep(8)

# Verificar que el servidor está corriendo
server_ok = False
for attempt in range(3):
    try:
        response = requests.get('http://localhost:8080/api/cases', timeout=5)
        print("✅ Servidor Flask iniciado correctamente en puerto 8080")
        server_ok = True
        break
    except:
        if attempt < 2:
            print(f"⏳ Intento {attempt + 1}/3 - Esperando servidor...")
            time.sleep(5)

# PASO 4: Obtener URL de ngrok (método alternativo: parsear logs)
public_url = None

# Método 1: Intentar API de ngrok
for attempt in range(3):
    try:
        response = requests.get('http://localhost:4040/api/tunnels', timeout=3)
        tunnels = response.json()['tunnels']
        if tunnels:
            public_url = tunnels[0]['public_url']
            break
    except:
        time.sleep(2)

# Método 2: Si API falla, parsear logs de ngrok
if not public_url:
    print("⏳ Método API falló, parseando logs de ngrok...")
    time.sleep(3)
    try:
        with open('/tmp/ngrok.log', 'r') as f:
            log_content = f.read()
            # Buscar URL en formato: url=https://xxxx.ngrok-free.app
            match = re.search(r'url=(https://[^\s]+\.ngrok[^\s]*)', log_content)
            if match:
                public_url = match.group(1)
    except Exception as e:
        print(f"⚠️ Error parseando logs: {e}")

if public_url:
    print("\n" + "="*70)
    print("✅ ✅ ✅  SERVIDOR LISTO  ✅ ✅ ✅")
    print("="*70)
    print(f"\n🌍 URL Pública: {public_url}")
    print("\n📋 Instrucciones:")
    print("   1. Haz click en el botón de abajo")
    print("   2. ¡Comienza a usar SimuPaciente!")
    print("\n⚠️  IMPORTANTE: NO detengas esta celda mientras uses la app")
    print("="*70)
    
    # Mostrar enlace clickable
    display(HTML(f'''
        <div style="
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 30px;
            border-radius: 15px;
            margin: 20px 0;
            text-align: center;
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        ">
            <h2 style="color: white; margin-bottom: 20px; font-size: 24px;">🏥 SimuPaciente UMH</h2>
            <a href="{public_url}" target="_blank" style="
                display: inline-block;
                background: white;
                color: #667eea;
                padding: 15px 40px;
                text-decoration: none;
                border-radius: 25px;
                font-weight: bold;
                font-size: 18px;
                box-shadow: 0 5px 15px rgba(0,0,0,0.2);
            ">
                🚀 Abrir Aplicación
            </a>
            <p style="color: white; margin-top: 20px; font-size: 14px;">
                {public_url}
            </p>
        </div>
    '''))
else:
    print("\n❌ No se pudo obtener la URL de ngrok")
    print("\nIntenta lo siguiente:")
    print("   1. Detén esta celda (botón Stop)")
    print("   2. Espera 10 segundos")
    print("   3. Ejecuta esta celda nuevamente")

# Mantener servidor corriendo y mostrar logs
print("\n📊 Logs del servidor:")
print("-" * 70)

try:
    for line in server_process.stdout:
        print(line.rstrip())
except KeyboardInterrupt:
    print("\n🛑 Deteniendo servidor...")
    server_process.terminate()
    ngrok_process.terminate()
    ngrok_log_file.close()
    print("✅ Servidor detenido")

---

## 📝 Notas Adicionales

### 🔄 Reiniciar el Servidor
Si necesitas reiniciar:
1. Detén la celda anterior (botón Stop)
2. Ejecuta la celda nuevamente

### 🐛 Solución de Problemas

**Problema:** No se conecta a OpenAI Realtime API
- Verifica que tu API key tiene acceso a Realtime API (beta)
- Comprueba que tienes créditos disponibles

**Problema:** La URL de ngrok no funciona
- Espera 30 segundos y recarga la página
- Verifica que la celda del servidor sigue corriendo [*]

### 📧 Soporte
- GitHub: https://github.com/marcosbenghezala/ECOE
- Universidad Miguel Hernández de Elche

---

**Versión:** 0.9.5 (Release Candidate)  
**Última actualización:** 17 de diciembre de 2025